In [ ]:
!curl -O https://download.pytorch.org/tutorial/data.zip; unzip data.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2814k  100 2814k    0     0  8296k      0 --:--:-- --:--:-- --:--:-- 8302k
Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/Italian.txt  
  inflating: data/names/Japanese.txt  
  inflating: data/names/Korean.txt   
  inflating: data/names/Polish.txt   
  inflating: data/names/Portuguese.txt  
  inflating: data/names/Russian.txt  
  inflating: data/names/Scottish.txt  
  inflating: data/names/Spanish.txt  
  inflating

In [ ]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.5 MB/s eta 0:00:00


In [ ]:
import os
import random
from string import ascii_letters

import torch
from torch import nn
import torch.nn.functional as F
from unidecode import unidecode

_ = torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data_dir = "./data/names"

lang2label = {
    file_name.split(".")[0]: torch.tensor([i], dtype=torch.long)
    for i, file_name in enumerate(os.listdir(data_dir))
}

In [ ]:
lang2label

{'Russian': tensor([0]),
 'Spanish': tensor([1]),
 'Portuguese': tensor([2]),
 'Arabic': tensor([3]),
 'Greek': tensor([4]),
 'German': tensor([5]),
 'French': tensor([6]),
 'Korean': tensor([7]),
 'Dutch': tensor([8]),
 'Irish': tensor([9]),
 'Scottish': tensor([10]),
 'Italian': tensor([11]),
 'Polish': tensor([12]),
 'Chinese': tensor([13]),
 'Czech': tensor([14]),
 'Vietnamese': tensor([15]),
 'English': tensor([16]),
 'Japanese': tensor([17])}

In [ ]:
num_langs = len(lang2label)

In [ ]:
unidecode("Ślusàrski")

'Slusarski'

In [ ]:
char2idx = {letter: i for i, letter in enumerate(ascii_letters + " .,:;-'")}
num_letters = len(char2idx); num_letters

59

In [ ]:
def name2tensor(name):
    tensor = torch.zeros(len(name), 1, num_letters)
    for i, char in enumerate(name):
        tensor[i][0][char2idx[char]] = 1
    return tensor

In [ ]:
name2tensor("Aqsa")

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       

In [ ]:
tensor_names = []
target_langs = []

for file in os.listdir(data_dir):
    with open(os.path.join(data_dir, file)) as f:
        lang = file.split(".")[0]
        names = [unidecode(line.rstrip()) for line in f]
        for name in names:
            try:
                tensor_names.append(name2tensor(name))
                target_langs.append(lang2label[lang])
            except KeyError:
                pass

In [ ]:
from sklearn.model_selection import train_test_split

train_idx, test_idx = train_test_split(
    range(len(target_langs)),
    test_size = 0.1,
    shuffle = True,
    stratify = target_langs
)

train_dataset = [
    (tensor_names[i], target_langs[i])
    for i in train_idx
]

test_dataset = [
    (tensor_names[i], target_langs[i])
    for i in test_idx
]

In [ ]:
len(train_dataset); len(test_dataset)

2007

Simple RNN

In [ ]:
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyRNN, self).__init__()
        self.hidden_size = hidden_size
        self.in2hidden = nn.Linear(input_size + hidden_size, hidden_size)
        self.in2output = nn.Linear(input_size + hidden_size, output_size)

    def forward(self, x, hidden_state):
        combined = torch.cat((x, hidden_state), 1)
        hidden = torch.sigmoid(self.in2hidden(combined))
        output = self.in2output(combined)
        return output, hidden

    def init_hidden(self):
        return nn.init.kaiming_uniform_(torch.empty(1, self.hidden_size))

In [ ]:
hidden_size = 256
learning_rate = 0.001

model = MyRNN(num_letters, hidden_size, num_langs)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
num_epochs = 10
print_interval = 6000

for epoch in range(num_epochs):
    random.shuffle(train_dataset)
    for i, (name, label) in enumerate(train_dataset):
        hidden_state = model.init_hidden()
        for char in name:
            output, hidden_state = model(char, hidden_state)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()

        if (i + 1) % print_interval == 0:
            print(
                f"Epoch [{epoch + 1}/{num_epochs}], "
                f"Step [{i + 1}/{len(train_dataset)}], "
                f"Loss: {loss.item():.4f}"
            )

Epoch [1/10], Step [6000/18063], Loss: 2.4170
Epoch [1/10], Step [12000/18063], Loss: 0.6860
Epoch [1/10], Step [18000/18063], Loss: 6.1761
Epoch [2/10], Step [6000/18063], Loss: 6.3844
Epoch [2/10], Step [12000/18063], Loss: 0.0000
Epoch [2/10], Step [18000/18063], Loss: 2.7758
Epoch [3/10], Step [6000/18063], Loss: 0.1898
Epoch [3/10], Step [12000/18063], Loss: 0.0033
Epoch [3/10], Step [18000/18063], Loss: 0.1699
Epoch [4/10], Step [6000/18063], Loss: 0.0042
Epoch [4/10], Step [12000/18063], Loss: 6.3242
Epoch [4/10], Step [18000/18063], Loss: 0.0284
Epoch [5/10], Step [6000/18063], Loss: 0.0142
Epoch [5/10], Step [12000/18063], Loss: 0.0775
Epoch [5/10], Step [18000/18063], Loss: 0.0000
Epoch [6/10], Step [6000/18063], Loss: 0.0269
Epoch [6/10], Step [12000/18063], Loss: 0.0000
Epoch [6/10], Step [18000/18063], Loss: 3.9840
Epoch [7/10], Step [6000/18063], Loss: 2.3213
Epoch [7/10], Step [12000/18063], Loss: 5.7301
Epoch [7/10], Step [18000/18063], Loss: 1.8057
Epoch [8/10], Step [

In [ ]:
num_correct = 0
num_samples = len(test_dataset)

model.eval()
with torch.no_grad():
    for name, label in test_dataset:
        hidden_state = model.init_hidden()
        for char in name:
            output, hidden_state = model(char, hidden_state)
        _, pred = torch.max(output, dim=1)
        num_correct += bool(pred == label)

print(f"Accuracy: {num_correct / num_samples * 100:.4f}%")


Accuracy: 79.5217%


In [ ]:
label2lang = {label.item(): lang for lang, label in lang2label.items()}

def myrnn_predict(name):
    model.eval()
    tensor_name = name2tensor(name)
    with torch.no_grad():
        hidden_state = model.init_hidden()
        for char in tensor_name:
            output, hidden_state = model(char, hidden_state)
        _, pred = torch.max(output, dim=1)
    model.train()
    return label2lang[pred.item()]

In [ ]:
myrnn_predict("Khadija")           # model acts poor as it's not predicting well.

'Russian'

Pytorch GRU

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, num_layers, hidden_size):
        super(GRUModel, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(
            input_size=num_letters,
            hidden_size=hidden_size,
            num_layers=num_layers,
        )
        self.fc = nn.Linear(hidden_size, num_langs)

    def forward(self, x):
        hidden_state = self.init_hidden()
        output, hidden_state = self.gru(x, hidden_state)
        output = self.fc(output[-1])
        return output

    def init_hidden(self):
        return torch.zeros(self.num_layers, 1, self.hidden_size).to(device)

In [ ]:
model_1 = GRUModel(num_layers=2, hidden_size=hidden_size)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,)

In [ ]:
for epoch in range(num_epochs):
    random.shuffle(train_dataset)
    for i, (name, label) in enumerate(train_dataset):
        output = model_1(name)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % print_interval == 0:
            print(
                f"Epoch [{epoch + 1}/{10}], "
                f"Step [{i + 1}/{len(train_dataset)}], "
                f"Loss: {loss.item():.4f}",
            )

Epoch [1/10], Step [6000/18063], Loss: 2.8682
Epoch [1/10], Step [12000/18063], Loss: 2.8760
Epoch [1/10], Step [18000/18063], Loss: 2.8687
Epoch [2/10], Step [6000/18063], Loss: 2.8848
Epoch [2/10], Step [12000/18063], Loss: 2.8656
Epoch [2/10], Step [18000/18063], Loss: 2.8584
Epoch [3/10], Step [6000/18063], Loss: 2.8753
Epoch [3/10], Step [12000/18063], Loss: 2.8752
Epoch [3/10], Step [18000/18063], Loss: 2.8716
Epoch [4/10], Step [6000/18063], Loss: 2.7990
Epoch [4/10], Step [12000/18063], Loss: 2.8744
Epoch [4/10], Step [18000/18063], Loss: 2.8805
Epoch [5/10], Step [6000/18063], Loss: 2.8860
Epoch [5/10], Step [12000/18063], Loss: 2.8718
Epoch [5/10], Step [18000/18063], Loss: 2.8742
Epoch [6/10], Step [6000/18063], Loss: 2.8719
Epoch [6/10], Step [12000/18063], Loss: 2.8707
Epoch [6/10], Step [18000/18063], Loss: 2.8748
Epoch [7/10], Step [6000/18063], Loss: 2.9220
Epoch [7/10], Step [12000/18063], Loss: 2.9088
Epoch [7/10], Step [18000/18063], Loss: 2.8620
Epoch [8/10], Step [

In [ ]:
num_correct = 0

model.eval()

with torch.no_grad():
    for name, label in test_dataset:
        output = model_1(name)
        _, pred = torch.max(output, dim=1)
        num_correct += bool(pred == label)

print(f"Accuracy: {num_correct / num_samples * 100:.4f}%")

Accuracy: 1.4948%


In [ ]:
def pytorch_predict(name):
    model_1.eval()
    tensor_name = name2tensor(name)
    with torch.no_grad():
        output = model_1(tensor_name)
        _, pred = torch.max(output, dim=1)
    model_1.train()
    return label2lang[pred.item()]

In [ ]:
pytorch_predict("mqin")

'Spanish'

In [ ]:
# I guess we've had some trouble traininig the RNN which is Pytorch + GRU.... see the accuracy -- 1.4% -- :(((
# Let's give it a go next time :)